# Parallel Computing

In this lesson we will deal with parallel computing, which is a type of computation in which many calculations or the execution of processes are carried out simultaneously on different CPU cores. We will show the differences between multi-threading and multi-processing and we will learn how those techniques are implemented in Julia.

For this lesson you will need Julia version 1.3 or above.

## Contents

- [Intro](#Intro)
- [Timing](#Timing)
- [Data based](#Data-based)
- [Task based](#Task-based)


## Intro 
In order to use multi-threading we need to start Julia with a number of threads equal to the number of you CPU cores. If you are using the Juno IDE it will automatically start Julia with the appropriate number of threads. If you are working from the REPL, you need to manually start Julia from the command line.

```
$ export JULIA_NUM_THREADS=4
$ julia
```

In [ ]:
# Check number of threads
Threads.nthreads()